In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import gc
import datetime
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_validate
from sklearn.preprocessing import MinMaxScaler,LabelEncoder, StandardScaler, RobustScaler
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler

### Here we have disabled the warnings to avoid getting some warnings

In [ ]:
warnings.simplefilter("ignore")

### Adjustments have been made for the columns and columns to be displayed

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Importing data from Dataset

In [ ]:
train_transaction=pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv')
train_identity=pd.read_csv('../input/ieee-fraud-detection/train_identity.csv')
test_transaction=pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv')
test_identity=pd.read_csv('../input/ieee-fraud-detection/test_identity.csv')
sample_submission=pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')

### The data sets given separately were merged

In [ ]:
train_df = train_transaction.merge(train_identity, how="left", on="TransactionID")

test_df = test_transaction.merge(test_identity, how="left", on="TransactionID")

### The name mismatch between the Train and test datasets has been fixed.

In [ ]:
test_df= test_df.rename(columns=lambda x:"_".join(x.split("-")))

# Reduce Memory

### Changing data types to change the memory space of the data set

In [ ]:
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

# Useless From Eda

In [ ]:
train_df.drop(['M1','id_27','V1','V14','V41','V68','V305'],inplace=True,axis=1)
test_df.drop(['M1','id_27','V1','V14','V41','V68','V305'],inplace=True,axis=1)

# Some Outliers

### Dropping some outlier variables from dataset

In [ ]:
droptamt = train_df[train_df['TransactionAmt']>10000].index
train_df.drop(droptamt, inplace=True)

dropdist1 = train_df[train_df['dist1']>6000].index
train_df.drop(dropdist1, inplace=True)

dropdist2 = train_df[train_df['dist2']>8000].index
train_df.drop(dropdist2, inplace=True)

dropc1 = train_df[train_df['C1'] > 2000].index
train_df.drop(dropc1, inplace=True)

dropc2 = train_df[train_df['C2'] > 2000].index
train_df.drop(dropc2, inplace=True)

#dropping the -ve values
dropd4 = train_df[train_df['D4']<0].index
train_df.drop(dropd4, inplace=True)

dropd6 = train_df[train_df['D6']<0].index
train_df.drop(dropd6, inplace=True)

dropd11 = train_df[train_df['D11']<0].index
train_df.drop(dropd11, inplace=True)

dropd12 = train_df[train_df['D12']<0].index
train_df.drop(dropd12, inplace=True)

dropd14 = train_df[train_df['D14']<0].index
train_df.drop(dropd14, inplace=True)

dropd15 = train_df[train_df['D15']<0].index
train_df.drop(dropd15, inplace=True)

# Filling Missing Values

### Showing missing values ratio

In [ ]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")
    
    if na_name:
        return na_columns,missing_df

In [ ]:
na_cols,missing_df=missing_values_table(train_df, True)
missing_df.reset_index(inplace=True)

# Feature Engineering

### Creating new variables

In [ ]:
def feature_eng(dataframe,name=True):
    
    dataframe['hour'] = (dataframe['TransactionDT']/(3600))%24
    
    def hourFeature(hour):
        if hour > 3 and hour < 12:
            return "highalert"
        if hour > 18 and hour <= 23:
            return "mediumalert"
        else:
            return "lowalert"
        
    dataframe['alertFeature'] = dataframe['hour'].apply(hourFeature)
    
    dataframe['LogTransactionAmt'] = np.log(dataframe['TransactionAmt'])
    
    
    
    
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]
    if name:
        dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
        dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

        dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
        dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

        dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
        dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    dataframe['had_id'] = "1"
    gc.collect()
    
    dataframe['c2_check']=dataframe["card2"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['c3_check']=dataframe["card3"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe["new_domain"]=dataframe['P_emaildomain'].apply(lambda x:str(x).partition(".")[0])
    dataframe['t_dt_minute']=dataframe["TransactionDT"]/60
    dataframe['t_dt_hour']=dataframe["t_dt_minute"]/60
    dataframe['t_dt_day']=dataframe["t_dt_hour"]/24
    dataframe['t_dt_week']=dataframe["t_dt_day"]/7

    # ADD MONTH FEATURE
    START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
    dataframe['DT_M'] = dataframe['TransactionDT'].map(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
    dataframe['DT_M'] = (dataframe['DT_M'].dt.year-2017)*12 + dataframe['DT_M'].dt.month 

    dataframe['card1check']=dataframe["card1"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['card2check']=dataframe["card2"].apply(lambda x: 0 if np.isnan(x)  else 1)

    dataframe['D2_check']=dataframe["D2"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D3_check']=dataframe["D3"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D4_check']=dataframe["D4"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D5_check']=dataframe["D5"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D6_check']=dataframe["D6"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D7_check']=dataframe["D7"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['d8_check']=dataframe["D8"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D9_check']=dataframe["D9"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D10_check']=dataframe["D10"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D11_check']=dataframe["D11"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D12_check']=dataframe["D12"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D13_check']=dataframe["D13"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D14_check']=dataframe["D14"].apply(lambda x: 0 if np.isnan(x)  else 1)
    dataframe['D15_check']=dataframe["D15"].apply(lambda x: 0 if np.isnan(x)  else 1)

    dataframe["new_domain1"]=dataframe['P_emaildomain'].apply(lambda x:str(x).partition(".")[-1])
    dataframe["new_domain2"]=dataframe['R_emaildomain'].apply(lambda x:str(x).partition(".")[-1])
    
    dataframe["addr1_addr2"] = dataframe["addr1"] + dataframe["addr2"]
    dataframe["card1_card2"] = dataframe["card1"] + dataframe["card2"]
    
    dataframe['TransactionAmt_to_mean_card1'] = dataframe['TransactionAmt'] / dataframe.groupby(['card1'])['TransactionAmt'].transform('mean')
    dataframe['TransactionAmt_to_mean_card4'] = dataframe['TransactionAmt'] / dataframe.groupby(['card4'])['TransactionAmt'].transform('mean')
    dataframe['TransactionAmt_to_std_card1'] = dataframe['TransactionAmt'] / dataframe.groupby(['card1'])['TransactionAmt'].transform('std')
    dataframe['TransactionAmt_to_std_card4'] = dataframe['TransactionAmt'] / dataframe.groupby(['card4'])['TransactionAmt'].transform('std')

    dataframe['id_02_to_mean_card1'] = dataframe['id_02'] / dataframe.groupby(['card1'])['id_02'].transform('mean')
    dataframe['id_02_to_mean_card4'] = dataframe['id_02'] / dataframe.groupby(['card4'])['id_02'].transform('mean')
    dataframe['id_02_to_std_card1'] = dataframe['id_02'] / dataframe.groupby(['card1'])['id_02'].transform('std')
    dataframe['id_02_to_std_card4'] = dataframe['id_02'] / dataframe.groupby(['card4'])['id_02'].transform('std')

    dataframe['D15_to_mean_card1'] = dataframe['D15'] / dataframe.groupby(['card1'])['D15'].transform('mean')
    dataframe['D15_to_mean_card4'] = dataframe['D15'] / dataframe.groupby(['card4'])['D15'].transform('mean')
    dataframe['D15_to_std_card1'] = dataframe['D15'] / dataframe.groupby(['card1'])['D15'].transform('std')
    dataframe['D15_to_std_card4'] = dataframe['D15'] / dataframe.groupby(['card4'])['D15'].transform('std')

    dataframe['D15_to_mean_addr1'] = dataframe['D15'] / dataframe.groupby(['addr1'])['D15'].transform('mean')
    dataframe['D15_to_std_addr1'] = dataframe['D15'] / dataframe.groupby(['addr1'])['D15'].transform('std')
    
    return dataframe

In [ ]:
train_df = feature_eng(train_df)

In [ ]:
test_df = feature_eng(test_df)

# Drop Useless Features

In [ ]:
useless = ['V22','V27','V28','V31', 'V34', 'V100', 'V39', 'V125', 'V122', 'V50', 'V119', 'V118', 'V117', 'V116', 'V114', 'V113', 'V57',
           'V58', 'V110', 'V65', 'V107', 'V71', 'V21', 'V18', 'V16', 'V89', 'V88', 'V321', 'V93']

In [ ]:
reduce = [col for col in train_df.columns if col not in useless] 

In [ ]:
reduce_test = [col for col in train_df.columns if col not in useless and col not in ["isFraud"]] 

In [ ]:
train_df = train_df[reduce]

In [ ]:
test_df = test_df[reduce_test]

# Converting Infinity Values to Nan

In [ ]:
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop variables that are missing more than 60% 

In [ ]:
drop_variables=missing_df[missing_df["ratio"]>60]['index']

In [ ]:
train_df.drop(drop_variables,inplace=True,axis=1)


In [ ]:
test_df.drop(drop_variables,inplace=True,axis=1)

# Filling Missing Values


### Filling missing values with -999

In [ ]:
train_df = train_df.fillna(-999)
test_df = test_df.fillna(-999)

### Checking missing values

In [ ]:
train_df.isnull().sum()

# Label Encoder

### Using Label Encoder to make the data suitable for the machine learning model.

In [ ]:
for f in train_df.columns:
    if train_df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train_df[f].values))
        train_df[f] = lbl.transform(list(train_df[f].values)) 

In [ ]:
for f in test_df.columns:
    if test_df[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(test_df[f].values))
        test_df[f] = lbl.transform(list(test_df[f].values))

# Reduce Memory Again

### Since new variables were produced, data reduction was done again.

In [ ]:
train_df = reduce_mem_usage(train_df)

In [ ]:
test_df = reduce_mem_usage(test_df)

# Creating models

In [ ]:
y = train_df["isFraud"]
X = train_df.drop(["isFraud", "TransactionID"], axis=1)
test_df = test_df.drop(["TransactionID"], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

# Random Under Sampling

In [ ]:
print("Before UnderSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before UnderSampling, counts of label '0': {} \n".format(sum(y_train==0)))

undersample = RandomUnderSampler(sampling_strategy=0.2, random_state=3)

X_train_res, y_train_res = undersample.fit_resample(X, y)

print('After UnderSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After UnderSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After UnderSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After UnderSampling, counts of label '0': {}".format(sum(y_train_res==0)))

## Modeling LightGBM

### Using best parameters

In [ ]:
model_lgbm = LGBMClassifier(objective='binary',
                    boosting_type='gbdt',
                    metric='auc',
                    n_jobs=-1,
                    learning_rate=0.01,
                    num_leaves= 2**8,
                    max_depth=-1,
                    tree_learner='serial',
                    colsample_bytree= 0.7,
                    subsample_freq=1,
                    subsample=0.7,
                    n_estimators=800,
                    max_bin=255,
                    verbose=-1) 
model_lgbm.fit(X_train_res,y_train_res) 

In [ ]:
pred3 = model_lgbm.predict(X_test)  
print(classification_report(y_test, pred3))
print(f"Auc: {round(roc_auc_score(pred3, y_test), 2)}")

In [ ]:
def plot_importance(model, features, num=len(X), save=False):

    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:50])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')
        
        


In [ ]:
plot_importance(model_lgbm, X_train)

In [ ]:
predictions3 = model_lgbm.predict_proba(test_df)[:,1]

In [ ]:
submission = pd.DataFrame({'TransactionID':test_transaction['TransactionID'],'isFraud':predictions3})
submission["TransactionID"]=submission["TransactionID"].astype(int)
submission.head()

### Saving submission file

In [ ]:
filename = 'CIS Fraud Detection under sampling lgbm.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

In [ ]:
feature_imp = pd.DataFrame({'Value': model_lgbm.feature_importances_, 'Feature': X_train.columns}).sort_values(by="Value")
feature_imp

In [ ]:
# Score: 0.909003
# Public score: 0.938276

## Modeling XGBoost

### Using best parameters

In [ ]:
model_xgb = XGBClassifier(n_estimators=500,
              max_depth=9,
              learning_rate=0.05,
              subsample=0.9,
              colsample_bytree=0.9,
              gamma = 0.2,
              alpha = 4,
              tree_method='hist')  
model_xgb.fit(X_train_res,y_train_res) 

In [ ]:
pred4 = model_xgb.predict(X_test)  
print(classification_report(y_test, pred4))
print(f"Auc: {round(roc_auc_score(pred4, y_test), 4)}")

In [ ]:
plot_importance(model_xgb, X_train)

In [ ]:
predictions4 = model_xgb.predict_proba(test_df)[:,1]

In [ ]:
submission = pd.DataFrame({'TransactionID':test_transaction['TransactionID'],'isFraud':predictions4})
submission["TransactionID"]=submission["TransactionID"].astype(int)
submission.head()

### Saving submission file

In [ ]:
filename = 'CIS Fraud Detection under sampling xgb.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

In [ ]:
feature_imp = pd.DataFrame({'Value': model_xgb.feature_importances_, 'Feature': X_train.columns}).sort_values(by="Value")
feature_imp

In [ ]:
# Score: 0.906771
# Public score: 0.936787